#**CHAP7. 앙상블 학습과 랜덤 포레스트**

##**1. 투표 기반 분류기**
* 여러 개 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측 (다수결 투표, 직접 투표 분류기)
* 앙상블 방법은 예측기가 가능한 한 서로 독립적일 때 최고의 성능을 발휘함
  * 각각 다른 알고리즘으로 분류기를 학습시키면 매우 다른 종류의 오차를 만들 가능성이 높기 때문에 앙상블 모델의 정확도를 향상시킬 수 있음


In [3]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

X,y=make_moons(n_samples=100, noise=0.15)
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.3)


In [4]:
# 투표 기반 분류기
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()

voting_clf=VotingClassifier(
    estimators=[('lr', log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [5]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test,y_pred))

LogisticRegression 0.9
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


> 투표 기반 분류기가 개별 분류기보다 성능이 조금 더 높음

* 모든 분류기가 클래스의 확률을 예측할 수 있으면 (=predict_proba()메서드가 있으면) 개별 분류기의 예측을 평균내어 확률이 가장 높은 클래스를 예측할 수 있음  > 간접 투표
  * 확률이 높은 투표에 비중
  * voting='soft'

##**2. 배깅과 페이스팅**

* 배깅과 페이스팅
  * 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각각 다르게 학습시킴
  * 배깅: 중복 허용 샘플링 / 페이스팅 : 중복 비허용 샘플링 
  * 투표 결과 수집함수는 분류일때는 mode (다수결), 회귀일때는 평균
  * 병렬 학습 가능

###1) 사이킷런의 배깅과 페이스팅


In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf=BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500, max_samples=50, bootstrap=True, n_jobs=-1
)
bag_clf.fit(X_train,y_train)
y_pred=bag_clf.predict(X_test)



* 앙상블의 예측이 결정트리 하나의 예측보다 일반화가 훨씬 잘 됨

* 앙상블은 비슷한 편향에서 더 작은 분산을 만듦

###2) oob 평가


* 배깅을 사용하면 어떤 샘플은 여러 번 샘플링되고 또 다른 어떤 샘플은 전혀 샘플링되지 않을 수 있음

* oob 샘플 : 훈련에 전혀 선택되지 않은 샘플

* 예측기가 훈련되는 동안 oob샘플을 전혀 사용하지 않으므로 별도의 검증 세트를 사용하지 않고 oob샘플을 사용해 평가할 수 있음

In [7]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500, bootstrap=True, n_jobs=-1, oob_score=True
)

bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

0.9285714285714286

In [8]:
y_pred=bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

1.0

In [9]:
# oob샘플에 대한 결정함수의 값
bag_clf.oob_decision_function_

array([[1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.47395833, 0.52604167],
       [1.        , 0.        ],
       [0.96954315, 0.03045685],
       [0.80102041, 0.19897959],
       [0.85164835, 0.14835165],
       [1.        , 0.        ],
       [0.02994012, 0.97005988],
       [0.4591195 , 0.5408805 ],
       [0.        , 1.        ],
       [0.84699454, 0.15300546],
       [0.01515152, 0.98484848],
       [0.32934132, 0.67065868],
       [0.02061856, 0.97938144],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.97849462, 0.02150538],
       [0.14438503, 0.85561497],
       [0.96756757, 0.03243243],
       [0.56284153, 0.43715847],
       [0.96969697, 0.03030303],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.25405405, 0.74594595],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.

##**3.랜덤 패치와 랜덤 서브스페이스**


* BaggingClassifier는 피처 샘플링도 지원
  * max_features, boostrap_features
  * 각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련됨

* 랜덤패치방식 : 피처와 샘플을 모두 샘플링
* 랜덤서브스페이스방식 : 샘플을 모두 사용하고 피처는 샘플링

* 피처 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춤

##**4. 랜덤포레스트**


* 배깅 방법을 적용한 결정트리앙상블

* max_samples를 훈련 세트의 크기로 지정

* RandomForestClassifier는 DecisionTreeClassifier의 매개변수와 BaggingClassifier의 매개변수를 모두 가지고 있음

* 랜덤포레스트 알고리즘은 트리의 노드르 분할 할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최선의 특성을 선택함
  * 트리를 다양하게 만들어 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 나은 모델을 만듦

In [10]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf=RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train,y_train)

y_pred_rf=rnd_clf.predict(X_test)

In [11]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(max_features="auto",max_leaf_nodes=16), n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

###1) 엑스트라 트리


* 후보 피처를 사용해 무작위로 분할한 다음 그 중에서 최상의 분할을 선택

* 익스트림 랜덤 트리 앙상블 (엑스트라 트리)


* RandomForest와 ExtraTree 중에 뭐가 더 나을지는 해봐야 알 수 있음 (교차검증으로 비교)

###2) 특성 중요도(피처 중요도)

* 랜덤포레스트의 장점 : 특성 중요도를 측정하기 쉬움
* 사이킷런은 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지로 특성 중요도를 측정

In [12]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf=RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
  print(name,score)

sepal length (cm) 0.10234729295977063
sepal width (cm) 0.023589449968693724
petal length (cm) 0.42585812671949247
petal width (cm) 0.44820513035204324


##**5.부스팅**


* 약한 학습기를 여러개 연결하여 강한 학습기를 만드는 앙상블 방법

* 앞의 모델을 보완해나가면서 일련의 예측기를 학습시킴

###1) 에이다부스트


* 이전 모델이 과소적합했던 훈련샘플의 가중치를 높임 >> 학습하기 어려운 샘플에 점점 더 맞춰짐

* 연속된 학습 기법은 병렬화를 할 수 없음

* 모든 예측기의 예측을 계산하고 예측기 가중치를 더해 예측 결과를 만듦 > 가중치 합이 가장 큰 클래스가 예측 결과가 됨

In [13]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=3),n_estimators=200, algorithm="SAMME.R", learning_rate=0.5
)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3),
                   learning_rate=0.5, n_estimators=200)

###2) 그래디언트 부스팅

* 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가함
* 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킴

In [14]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [15]:
#첫번째 예측기에서 생긴 잔여 오차에 두번째 DecisionTreeRegressor를 훈련시킴
y2=y-tree_reg1.predict(X)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [16]:
# 두번째 예측기에서 생긴 잔여 오차에 세번째 회귀 모델을 훈련시킴
y3=y2-tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [17]:
# 모든 트리의 예측을 더해 새로운 샘플에 대한 예측 만들기
y_pred=sum(tree.predict(X) for tree in (tree_reg1, tree_reg2, tree_reg3))

사이킷런의 GradientBoostingRegressor사용

In [18]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train,X_val,y_train,y_val=train_test_split(X,y)

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors=[mean_squared_error(y_val,y_pred)
for y_pred in gbrt.staged_predict(X_val)]

bst_n_estimators=np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train,y_train)

GradientBoostingRegressor(max_depth=2)

In [22]:
gbrt=GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error=float('inf')
error_going_up=0

for n_estimators in range(1,120):
  gbrt.n_estimators=n_estimators
  gbrt.fit(X_train,y_train)
  y_pred=gbrt.predict(X_val)
  val_error=mean_squared_error(y_val,y_pred)
  if val_error < min_val_error:
    min_val_error=val_error
    error_goin_up=0
  else:
    error_going_up +=1
    if error_going_up==5:
      break


* 무작위로 선택된 훈련샘플로 학습 > 확률적 그래디언트 부스팅 (XGBoost)

In [23]:
import xgboost
xgb_reg=xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
y_pred=xgb_reg.predict(X_val)

[07:24:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [24]:
xgb_reg.fit(X_train,y_train, eval_set=[(X_val,y_val)], early_stopping_rounds=2)
y_pred=xgb_reg.predict(X_val)

[07:24:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.467074
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.438712
[2]	validation_0-rmse:0.414482
[3]	validation_0-rmse:0.393967
[4]	validation_0-rmse:0.376765
[5]	validation_0-rmse:0.364486
[6]	validation_0-rmse:0.353693
[7]	validation_0-rmse:0.34528
[8]	validation_0-rmse:0.341425
[9]	validation_0-rmse:0.335924
[10]	validation_0-rmse:0.334351
[11]	validation_0-rmse:0.330551
[12]	validation_0-rmse:0.31965
[13]	validation_0-rmse:0.310539
[14]	validation_0-rmse:0.30862
[15]	validation_0-rmse:0.300743
[16]	validation_0-rmse:0.29788
[17]	validation_0-rmse:0.291228
[18]	validation_0-rmse:0.288816
[19]	validation_0-rmse:0.285037
[20]	validation_0-rmse:0.281936
[21]	validation_0-rmse:0.277414
[22]	validation_0-rmse:0.275758
[23]	validation_0-rmse:0.273735
[24]	validation_0-rmse:0.270319
[25]	validation_0-rmse:

##**6.스태킹**

* 앙상블에 속한 모든 예측기의 예측을 취합하는 모델을 훈련

* 각 예측기는 가각 다른 값을 예측하고 마지막 예측기(블렌더)가 이 예측을 입력으로 받아 최종 예측을 만듦

* 블렌더를 학습시키는 일반적인 방법 : 홀드 아웃

* 블렌더를 여러 개 훈련시키는 것도 가능

* 사이킷런은 스태킹을 직접 지원하지는 않음 (직접 구현)